# Text pre-processing

The main idea is to process company names as follows:
- Remove whitespace
- Normalize case
- Remove acronyms
- Remove accents
- Remove stop words

And finally save a single list of company names.

In [58]:
import re
import pandas as pd
import numpy as np

## Stop Words

Some capital letters are common to many names as they represent the commercial status of the company.

In [59]:
stop_words = [
    r"S\.?A\.? *DE\.? *C\.?V\.?",
    r"\bS\.?A\.?",
    r"S\.? *DE\.? *R\.?L\.? *DE *C\.?V\.?",
    r"S\.? *DE\.? *R\.?L\.?",
    r"S\.?A\.?P\.?I\.? *DE\.? *C\.?V\.?",
    r"\bS\.?A\.?P\.?I\.?\b",
    r"S\.A\.S\.?",
    r"S\.?A\.?B\.? *DE\.? *C\.?V\.?",
    r"\bC\.?V\.?\b"]
special_chars = [",", "."]
accents = ["á", "é", "í", "ó", "ú"]
vowels = ["a", "e", "i", "o", "u"]

In [60]:
more_stop_words = [
    r"\bcompany\b",
    r"\bcia\b",
    r"\bcompania\b",
    r"\bmarcas\b",
    r"\bmarca\b",
    r"(?<!de)\s+mexico",
    r"\bmex\b",
    r"\bmx\b",
    r"\busa\b",
    r"\bus\b",
    r"\bcomercializadora\b",
    r"\bindustrial\b",
    r"\bgrupo\b",
    r"\bgroup\b"
]

In [61]:
def clean_string(word):
    """Pre-process the company name"""
    
    # Remove main stop words
    for stop_word in stop_words:
        word = re.sub(stop_word, "", word, flags=re.I)
    # Remove spetial chars
    for spetial_char in special_chars:
        word = word.replace(spetial_char, "")
    # Remove accents
    for i, accent in enumerate(accents):
        word = re.sub(accent, vowels[i], word, flags=re.I)
    # Additional stop words
    for stop_word in more_stop_words:
        word = re.sub(stop_word, "", word, flags=re.I)
    word = word.strip()
    word = re.sub(r" +", " ", word)
    word = word.lower()
    
    return word

In [62]:
cnts = pd.read_csv("/home/rdora/declaranet/data/tables/contratos.csv")
exp = pd.read_csv("/home/rdora/declaranet/data/tables/experience.csv")

/home/rdora/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
# Not intersted in experience or companies with no names
print(f"There are {cnts.proveedor_contratista.isna().sum():,} missing contracts")
print(f"There are {exp.INSTITUCION_EMPRESA.isna().sum():,} missing companies in experience")
print("-" * 20)
print(
    f"There are {cnts.shape[0]:,} contracts and\n"
    f"there are {exp.shape[0]:,} experience entries")

cnts = cnts.dropna(subset=['proveedor_contratista'])
exp = exp.dropna(subset=['INSTITUCION_EMPRESA'])

There are 2,579 missing contracts
There are 2,069 missing companies in experience
--------------------
There are 1,801,208 contracts and
there are 1,810,431 experience entries


In [64]:
# Some numbers before the filter
cnts_unique = set(cnts.proveedor_contratista.unique())
exp_unique = set(exp[exp.SECTOR=="PRIVADO"].INSTITUCION_EMPRESA.unique())

print("Before pre-processing")
print("-" * 20)
print(
    "Number of unique company names:\n"
    f"In contracts: {len(cnts_unique):,}\n"
    f"In experience: {len(exp_unique):,}")
print(
    f"Total number of unique companies: {len(cnts_unique | exp_unique):,}\n"
    f"Total number of companies in BOTH tables: {len(cnts_unique & exp_unique):,}")
print("-" * 20)
print("Top 10 companies")
print("-" * 20)
print("Contracts\n", cnts.proveedor_contratista.value_counts().head(10))
print("-" * 20)
print("Experience\n", exp[exp.SECTOR=="PRIVADO"].INSTITUCION_EMPRESA.value_counts().head(10))

Before pre-processing
--------------------
Number of unique company names:
In contracts: 261,318
In experience: 398,421
Total number of unique companies: 653,764
Total number of companies in BOTH tables: 5,975
--------------------
Top 10 companies
--------------------
Contracts
 SUMMA COMPANY, S.A. DE C.V.                            13653
MARCAS NESTLE, S.A. DE C.V.                             9243
CIA. PROCTER & GAMBLE MEXICO, S. DE R.L. DE C.V.        7959
COMERCIALIZADORA PEPSICO MEXICO, S. DE R.L. DE C.V.     6952
FARMACEUTICOS MAYPO, S.A. DE C.V.                       6556
CONSERVAS LA COSTENA, S.A. DE C.V.                      5771
GRUPO FARMACOS ESPECIALIZADOS SA DE CV                  5611
GRUPO INDUSTRIAL MASECA, S.A.B. DE C.V.                 5556
UNILEVER DE MEXICO, S. DE R.L. DE C.V.                  5102
ALEN DEL NORTE SA DE CV                                 4859
Name: proveedor_contratista, dtype: int64
--------------------
Experience
 DESPACHO JURIDICO                  

In [65]:
# Apply the new filter
exp['INSTITUCION_EMPRESA'] = exp['INSTITUCION_EMPRESA'].apply(clean_string)

cnts['proveedor_contratista'] = cnts['proveedor_contratista'].apply(clean_string)

In [66]:
# Some numbers before the filter
cnts_unique = set(cnts.proveedor_contratista.unique())
exp_unique = set(exp[exp.SECTOR=="PRIVADO"].INSTITUCION_EMPRESA.unique())

print("After pre-processing")
print("-" * 20)
print(
    "Number of unique company names:\n"
    f"In contracts: {len(cnts_unique):,}\n"
    f"In experience: {len(exp_unique):,}")
print(
    f"Total number of unique companies: {len(cnts_unique | exp_unique):,}\n"
    f"Total number of companies in BOTH tables: {len(cnts_unique & exp_unique):,}")
print("-" * 20)
print("Top 10 companies")
print("-" * 20)
print("Contracts\n", cnts.proveedor_contratista.value_counts().head(10))
print("-" * 20)
print("Experience\n", exp[exp.SECTOR=="PRIVADO"].INSTITUCION_EMPRESA.value_counts().head(10))

After pre-processing
--------------------
Number of unique company names:
In contracts: 226,929
In experience: 313,766
Total number of unique companies: 526,562
Total number of companies in BOTH tables: 14,133
--------------------
Top 10 companies
--------------------
Contracts
 summa                         16590
nestle                        11712
fabrica de jabon la corona    10809
alen del norte                 9043
procter & gamble               9016
farmaceuticos maypo            7034
pepsico                        6953
farmacos especializados        6332
molinera de mexico             6065
conservas la costena           5771
Name: proveedor_contratista, dtype: int64
--------------------
Experience
 despacho juridico                  4247
petroleos mexicanos                1597
banamex                            1595
despacho contable                  1504
banco nacional de mexico           1100
bancomer sa                        1062
televisa                           1058
banco

## Identification of acronyms

La tabla de experiencia, que fue tomada de muchas más fuentes, podría tener una mayor tendencia a contener acrónimos.

In [67]:
"The Hongkong and Shanghai Banking Corporation".lower()

'the hongkong and shanghai banking corporation'

In [68]:
exp.replace?

In [69]:
# Dependencias públicas
deps = {
    re.compile(r"\bissste\b"): 'instituto de seguridad y servicios sociales de los trabajadores del estado',
    re.compile(r"pemex"): 'petroleos mexicanos',
    re.compile(r"\bimss\b"): 'instituto mexicano del seguro social',
    re.compile(r"\bsat\b"): 'servicio de administracion tributaria',
    re.compile(r"\bcfe\b"): 'comision federal de electricidad',
    re.compile(r"\bsre\b"): 'secretaria de relaciones exteriores',
    re.compile(r'\bsegob\b'): 'secretaria de gobernacion',
    re.compile(r"\bsep\b"): 'secretaria de educacion publica',
    re.compile(r"\bsfp\b"): 'secretaria de la funcion publica',
    re.compile(r"\binegi\b"): 'instituto nacional de estadistica geografia e informatica',
    re.compile(r"\bsagarpa\b"): 'secretaria de agricultura y desarrollo rural pesca y alimentacion',
    re.compile(r"\bconagua\b"): 'comision nacional del agua',
    re.compile(r"\bsedesol\b"): 'secretaria de desarrollo social',
    re.compile(r"\bsae\b"): 'servicio de administración y enajenación de bienes',
    re.compile(r"\bipn\b"): 'instituto politecnico nacional',
    re.compile(r"\bunam\b"): 'universidad nacional autonoma de mexico',
    re.compile(r"\buvm\b"): 'universidad del valle de mexico',
    re.compile(r"\bimp\b"): 'instituto mexicano del petroleo',
    re.compile(r"\bssp\b"): 'secretaria de seguridad publica',
    re.compile(r"\bpgr\b"): 'procuraduria federal de la republica',
    re.compile(r"\bife\b"): 'instituto federal electoral',
    re.compile(r"\bine\b"): 'instituto nacional electoral',
    re.compile(r"\binah\b"): 'instituto nacional de antropologia e historia',
    re.compile(r"\binmegen\b"): "instituto nacional de medicina genomica",
    re.compile(r"\btrife\b"): 'tribunal electoral del poder judicial de la federacion',
    re.compile(r"\btfjfa\b"): 'tribunal federal de justicia fiscal y administrativa',
    re.compile(r"\btelmex\b"): 'telefonos de mexico',
    re.compile(r"\bsemarnat\b"): 'secretaria   de   medio   ambiente   y   recursos naturales',
    re.compile(r"\bdiconsa\b"): 'distribuidora y comercializadora',
    re.compile(r"\bshcp\b"): 'secretaria de hacienda y credito publico',
    re.compile(r"\bprofeco\b"): 'procuraduria federal del consumidor',
    re.compile(r"\bconafo\b"): 'comision nacional forestal',
    re.compile(r"\bsedena\b"): 'secretaria de la defensa nacional',
    re.compile(r"\bfonatur\b"): 'fondo nacional de fomento al turismo',
    re.compile(r"\bconacyt\b"): 'consejo nacional de ciencia y tecnologia',
    re.compile(r"\bsni\b"): 'sistema nacional de investigadores',
    re.compile(r"\bconaculta\b"): 'consejo nacional para la cultura y las artes',
    re.compile(r"\bcnbv\b"): 'comision nacional bancaria y de valores',
    re.compile(r"\bsarh\b"): 'secretaria de agricultura y recursos hidraulicos',
    re.compile(r"\bbanxico\b"): 'banco de mexico',
    re.compile(r"\bse\b"): 'secretaria de economia',
    re.compile(r"\bpri\b"): 'partido revolucionario institucional',
    re.compile(r"\bpan\b"): 'partido accion nacional',
    re.compile(r"\bprd\b"): 'partido revolucionario democratico',
    re.compile(r"\bmorena\b"): 'movimiento de regeneracion nacional',
    re.compile(r"\bconalep\b"): 'colegio nacional de educacion profesional tecnica',
    re.compile(r"\binapam\b"): 'instituto nacional de las personas adultas mayores',
    re.compile(r"\binba\b"): 'instituto nacional de bellas artes',
    re.compile(r"\bcofetel\b"): 'comision federal de telecomunicaciones',
    re.compile(r"\bconapred\b"): 'consejo nacional para prevenir la discriminacion',
    re.compile(r"\binea\b"): "instituto nacional para la educacion de los adultos",
    re.compile(r"\buam\b"): "universidad autonoma metropolitana",
    re.compile(r"\bitam\b"): "instituto tecnologico autonomo de mexico",
    re.compile(r"\bitesm\b"): 'instituto tecnologico de estudios superiores de monterrey',
    re.compile(r"\bibero\b"): 'universidad iberoamericana',
    re.compile(r"\binfonavit\b"): 'instituto del fondo nacional de la vivienda para los trabajadores',
    re.compile(r"\bsagar\b"): "secretaria de agricultura ganaderia desarrollo rural pesca y alimentacion",
    re.compile(r"\bsenasica\b"): 'servicio nacional de sanidad inocuidad y calidad agroalimentaria',
    re.compile(r"\bsecodam\b"): 'secretaria de contraloria y desarrollo administrativo',
    re.compile(r"\bsct\b"): 'secretaria de comunicaciones y transportes',
    re.compile(r"\bcndh\b"): 'comision nacional de los derechos humanos',
    re.compile(r"\bpfp\b"): 'policia federal preventiva',
    re.compile(r"\bfce\b"): 'fondo de cultura economica',
    re.compile(r"\binifap\b"): 'instituto nacional de investigaciones forestales agricolas y pecuarias',
    re.compile(r"\bdif\b"): 'sistema nacional para el desarrollo integral de la familia',
    re.compile(r"\bssa\b"): 'secretaria de salud',
    re.compile(r"\bfonaes\b"): 'fondo nacional de apoyos para empresas en solidaridad'
}

In [70]:
deps_privadas = {
    re.compile(r"\bhsbc\b"): 'the hongkong and shanghai banking corporation',
    re.compile(r"\bbanamex\b"): 'banco nacional de mexico',
    re.compile(r"\bbancomer\b"): 'banco de comercio',
    re.compile(r"\bpemex\b"): 'petroleos mexicanos',
    re.compile(r"\biica\b"): 'instituto interamericano de cooperacion para la agricultura',
    re.compile(r"\bcomer\b"): "comercial mexicana",
    re.compile(r"\bunitec\b"): 'universidad tecnologica de mexico'}

In [71]:
deps.update(deps_privadas)

In [72]:
# Update acronyms
exp['INSTITUCION_EMPRESA'] = exp['INSTITUCION_EMPRESA'].replace(deps, regex=True)
cnts['proveedor_contratista'] = cnts['proveedor_contratista'].replace(deps, regex=True)

In [73]:
# Save tables
exp.to_csv("/home/rdora/declaranet/data/pre-process/experience.csv", index=False)
cnts.to_csv("/home/rdora/declaranet/data/pre-process/contracts.csv", index=False)

In [74]:
companies = np.array(list(set(exp['INSTITUCION_EMPRESA'].unique()) | set(cnts['proveedor_contratista'].unique())))

In [75]:
companies = companies[1:]

In [76]:
np.savetxt("/home/rdora/declaranet/data/pre-process/list_companies.txt", companies, fmt="%s", delimiter="\n")